# Exploracion del dataset de estaciones de Brasil usando PySpark

fuentes:
- https://sparkbyexamples.com/pyspark/pyspark-read-csv-file-into-dataframe/
- https://sparkbyexamples.com/pyspark/pyspark-where-filter/
- https://sparkbyexamples.com/pyspark/pyspark-dataframe-groupby-and-sort-by-descending-order/

In [1]:
import findspark
findspark.init()
findspark.find()

'D:\\apps\\spark-3.5.0-bin-hadoop3'

Inicializamos una sesion de spark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Perla Negra") \
                            .master("local[*]") \
                            .config("spark.ui.port", "4040") \
                            .getOrCreate()


Para leer un archivo CSV en un DataFrame de PySpark, usa `csv("path")` de DataFrameReader. Este artículo explora el proceso de lectura de archivos individuales, múltiples archivos o todos los archivos de un directorio local en un DataFrame utilizando PySpark.

Puntos clave:

- PySpark admite la lectura de archivos CSV con pipes, coma, tab, espacio u otros delimitadores/separadores.
- PySpark lee archivos CSV en paralelo, aprovechando múltiples nodos ejecutores para acelerar la ingestión de datos.
- PySpark puede inferir automáticamente el esquema de los archivos CSV, eliminando la necesidad de definir el esquema manualmente en muchos casos.
- Los usuarios tienen la flexibilidad de definir esquemas personalizados para los archivos CSV, especificando tipos de datos y nombres de columnas según sea necesario.
- PySpark ofrece opciones para manejar encabezados en los archivos CSV, permitiendo a los usuarios omitir los encabezados o tratarlos como filas de datos.
- Proporciona mecanismos robustos de manejo de errores para tratar con archivos CSV mal formateados o corruptos, asegurando la integridad de los datos.

In [5]:

# Read CSV File
df = spark.read.option("delimiter", ";").csv("datasets/brazil-weather/conventional_weather_stations_inmet_brazil_1961_2019.csv", )
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)



Aqui se leen los datos de las estaciones de brazil dentro de un DataFrame de Spark. _c0, _c1, etcetera son las columnas leidas en el dataset. Por defecto las columnas se tratan como strings.

In [10]:
# una manera alternativa de leer un archivo csv con Spark es usando format().load()

df = spark.read.option("delimiter", ";").format("csv") \
                  .load("datasets/brazil-weather/conventional_weather_stations_inmet_brazil_1961_2019.csv")
df.printSchema()


root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)



Si especificamos la opcion header=True, spark va a intentar encontrar los nombres de las columnas

In [14]:

# Read CSV File
df = spark.read.option("delimiter", ";") \
                        .option("header",True) \
                        .csv("datasets/brazil-weather/conventional_weather_stations_inmet_brazil_1961_2019.csv", )
df.printSchema()

root
 |-- Estacao: string (nullable = true)
 |-- Data: string (nullable = true)
 |-- Hora: string (nullable = true)
 |-- Precipitacao: string (nullable = true)
 |-- TempBulboSeco: string (nullable = true)
 |-- TempBulboUmido: string (nullable = true)
 |-- TempMaxima: string (nullable = true)
 |-- TempMinima: string (nullable = true)
 |-- UmidadeRelativa: string (nullable = true)
 |-- PressaoAtmEstacao: string (nullable = true)
 |-- PressaoAtmMar: string (nullable = true)
 |-- DirecaoVento: string (nullable = true)
 |-- VelocidadeVento: string (nullable = true)
 |-- Insolacao: string (nullable = true)
 |-- Nebulosidade: string (nullable = true)
 |-- Evaporacao Piche: string (nullable = true)
 |-- Temp Comp Media: string (nullable = true)
 |-- Umidade Relativa Media: string (nullable = true)
 |-- Velocidade do Vento Media: string (nullable = true)
 |-- _c19: string (nullable = true)



Tambien existe una opcion para inferir los tipos de datos de las columnas, aunque esto requiere leer una segunda vez el conjunto de datos. Esto ultimo es algo a considerar cuando trabajamos con datasets muy grandes y cada operacion es costosa. 

In [19]:
# pasamos las opciones al objeto DataFrameReader usando 'options' en lugar de 'option' para llamar una vez al metodo. Solo por ser una manera alternativa de llamarlo. No significa un costo menor. 
df = spark.read.options(delimiter=";", header=True, inferSchema=True) \
                        .csv("datasets/brazil-weather/conventional_weather_stations_inmet_brazil_1961_2019.csv", )
df.printSchema()

root
 |-- Estacao: integer (nullable = true)
 |-- Data: string (nullable = true)
 |-- Hora: integer (nullable = true)
 |-- Precipitacao: double (nullable = true)
 |-- TempBulboSeco: double (nullable = true)
 |-- TempBulboUmido: double (nullable = true)
 |-- TempMaxima: double (nullable = true)
 |-- TempMinima: double (nullable = true)
 |-- UmidadeRelativa: double (nullable = true)
 |-- PressaoAtmEstacao: double (nullable = true)
 |-- PressaoAtmMar: double (nullable = true)
 |-- DirecaoVento: double (nullable = true)
 |-- VelocidadeVento: double (nullable = true)
 |-- Insolacao: double (nullable = true)
 |-- Nebulosidade: double (nullable = true)
 |-- Evaporacao Piche: double (nullable = true)
 |-- Temp Comp Media: double (nullable = true)
 |-- Umidade Relativa Media: double (nullable = true)
 |-- Velocidade do Vento Media: double (nullable = true)
 |-- _c19: string (nullable = true)



In [22]:
# tambien se pueden especificar las opciones en un diccionario y pasar el parametro como **kwargs

# Define read options
options = {
    "inferSchema": "True",
    "delimiter": ";",
    "header":True
}

# Read a CSV file with specified options
df = spark.read.options(**options).csv("datasets/brazil-weather/conventional_weather_stations_inmet_brazil_1961_2019.csv")
df.printSchema()


root
 |-- Estacao: integer (nullable = true)
 |-- Data: string (nullable = true)
 |-- Hora: integer (nullable = true)
 |-- Precipitacao: double (nullable = true)
 |-- TempBulboSeco: double (nullable = true)
 |-- TempBulboUmido: double (nullable = true)
 |-- TempMaxima: double (nullable = true)
 |-- TempMinima: double (nullable = true)
 |-- UmidadeRelativa: double (nullable = true)
 |-- PressaoAtmEstacao: double (nullable = true)
 |-- PressaoAtmMar: double (nullable = true)
 |-- DirecaoVento: double (nullable = true)
 |-- VelocidadeVento: double (nullable = true)
 |-- Insolacao: double (nullable = true)
 |-- Nebulosidade: double (nullable = true)
 |-- Evaporacao Piche: double (nullable = true)
 |-- Temp Comp Media: double (nullable = true)
 |-- Umidade Relativa Media: double (nullable = true)
 |-- Velocidade do Vento Media: double (nullable = true)
 |-- _c19: string (nullable = true)



## Transformaciones de DataFrame
Las transformaciones de DataFrame en PySpark implican la aplicación de varias operaciones para manipular los datos dentro de un DataFrame. Estas transformaciones incluyen:

- Filtros: Seleccionar filas del DataFrame basadas en ciertas condiciones.
- Selección de Columnas: Extraer columnas específicas del DataFrame.
- Añadir Columnas: Crear nuevas columnas realizando cálculos o transformaciones en las columnas existentes.
- Eliminar Columnas: Eliminar columnas innecesarias del DataFrame.
- Agrupamiento y Agregación: Agrupar filas según ciertos criterios y calcular estadísticas agregadas, como suma, promedio, conteo, etc., dentro de cada grupo.
- Orden: Organizar las filas del DataFrame en un orden especificado basado en los valores de las columnas.
- Unión: Combinar dos DataFrames basándose en una clave o condición común.
- Unión Vertical: Concatenar dos DataFrames verticalmente, añadiendo filas de un DataFrame a otro.
- Pivoting & Melting: Reestructurar el DataFrame de formato largo a formato ancho (pivot) o de formato ancho a formato largo (melt).
- Funciones de ventana deslizante: Realizar cálculos sobre una ventana deslizante de filas, como calcular promedios móviles o rankings.

Es notable que todas estas funciones se pueden hacer con pandas y python. La necesidad aqui de usarlas en el contexto de PySpark es la posibilidad de trabajar estos metodos sobre RDDs y obteniendo un mejor rendimiento con datasets grandes.

## Ejemplos



### Operaciones de transformacion: `when`

Al igual que en SQL y en otros lenguajes de programación, PySpark admite una forma de verificar múltiples condiciones en secuencia y devuelve un valor cuando se cumple la primera condición, utilizando expresiones como case when en SQL y `when().otherwise()` en PySpark. Estas funcionan de manera similar a las declaraciones `switch` y `if then else`.

In [48]:
from pyspark.sql.functions import when
df2 = df.withColumn("TempBulboSeco", when(df.TempBulboSeco <= 10,"Baja") \
                                 .when(df.TempBulboSeco <= 20,"Moderada") \
                                 .when(df.TempBulboSeco > 20,"Alta"))
df2.show()

+-------+----------+----+------------+-------------+--------------+----------+----------+---------------+-----------------+-------------+------------+---------------+---------+------------+----------------+---------------+----------------------+-------------------------+----+
|Estacao|      Data|Hora|Precipitacao|TempBulboSeco|TempBulboUmido|TempMaxima|TempMinima|UmidadeRelativa|PressaoAtmEstacao|PressaoAtmMar|DirecaoVento|VelocidadeVento|Insolacao|Nebulosidade|Evaporacao Piche|Temp Comp Media|Umidade Relativa Media|Velocidade do Vento Media|_c19|
+-------+----------+----+------------+-------------+--------------+----------+----------+---------------+-----------------+-------------+------------+---------------+---------+------------+----------------+---------------+----------------------+-------------------------+----+
|  82024|01/01/1961|   0|        NULL|         NULL|          NULL|      32.3|      NULL|           NULL|             NULL|         NULL|        NULL|           NULL|   

### Operaciones de filtro: `filter` y `where`

Ahora filtremos el dataset para visualizar solo los datos de las estaciones etiquetados. Dejamos fuera los nulos

In [54]:
# Using equal condition
df2['Estacao', 'Data', 'TempBulboSeco', 'TempBulboUmido'] \
                                .filter(df2.TempBulboSeco.isNotNull()) \
                                .filter(df2.TempBulboUmido.isNotNull()) \
                                .filter(df2.TempBulboSeco == 'Baja').show()


+-------+----------+-------------+--------------+
|Estacao|      Data|TempBulboSeco|TempBulboUmido|
+-------+----------+-------------+--------------+
|  82024|21/08/1976|         Baja|          22.8|
|  82191|01/05/1970|         Baja|          25.5|
|  82331|20/05/1992|         Baja|           0.0|
|  82331|20/05/1992|         Baja|           6.5|
|  82331|21/05/1992|         Baja|           0.0|
|  83182|01/11/1996|         Baja|           0.0|
|  83182|01/11/1996|         Baja|           0.0|
|  83182|02/11/1996|         Baja|           0.0|
|  83264|18/07/1975|         Baja|           3.1|
|  83264|19/07/1981|         Baja|           5.8|
|  83264|09/06/1985|         Baja|           8.0|
|  83264|01/04/1986|         Baja|          23.0|
|  83264|07/07/1989|         Baja|           8.6|
|  83267|15/08/1999|         Baja|           5.9|
|  83267|13/07/2000|         Baja|           8.7|
|  83267|14/07/2000|         Baja|           7.7|
|  83270|18/05/1998|         Baja|          20.6|


### Ejercicio

Al transformar la columna `TempBulboSeco` estamos reduciendo y por lo tanto perdiendo informacion de la columna, pasando de un dato double a una categoria en string. Si bien esto puede ser deseable para agrupar, no hay razones por ahora para perder la columna original. En este ejercicio, van a crear una columna `TempBulboSeco_cat` que contenga la transformacion hecha mas arriba, y que conserve esa categoria, sin eliminar ni perder la columna original `TempBulboSeco`. 

## `groupBy`, `filter` y `sort`

Vamos a hacer un ejemplo de agrupacion, filtrado y ordenado sobre nuestro conjunto de datos. Supongamos que necesitamos saber la temperatura media de bulbo seco, por estacion. Podriamos hacer lo siguiente:

In [57]:
from pyspark.sql.functions import avg, col, desc
# Hacemos un promedio por estacion de la temperatura de bulbo seco
df.groupBy("Estacao") \
  .agg(avg("TempBulboSeco").alias("TempBulboSeco_avg")) \
  .filter(col("TempBulboSeco_avg") > 10)  \
  .sort(desc("TempBulboSeco_avg")) \
  .show()

+-------+------------------+
|Estacao| TempBulboSeco_avg|
+-------+------------------+
|  82690|30.127860210855143|
|  82296|29.659932405827835|
|  82474| 29.30886615176194|
|  82678|29.267836075752573|
|  82879|29.260869697894222|
|  82460|29.218999332229416|
|  82298|29.210521264272458|
|  82780| 29.11393250585912|
|  82975|29.087137417260166|
|  82870|29.035611291845285|
|  82480|28.966630503571423|
|  82791|28.950074794315977|
|  82476|  28.9276052987726|
|  82590|28.869825836093444|
|  82578|28.858674607196377|
|  82493| 28.84008219335511|
|  82693|28.707614601018964|
|  82588| 28.68681230116714|
|  82564| 28.56894557892424|
|  82392|28.565107830433384|
+-------+------------------+
only showing top 20 rows



In [60]:
# Mismo ejemplo usando Spark SQL
df.createOrReplaceTempView("EMP")
spark.sql("select Estacao, avg(TempBulboSeco) as TempBulboSeco_avg from EMP " +
          "group by Estacao having TempBulboSeco_avg > 10 " + 
          "order by TempBulboSeco_avg desc").show()

+-------+------------------+
|Estacao| TempBulboSeco_avg|
+-------+------------------+
|  82690|30.127860210855143|
|  82296|29.659932405827835|
|  82474| 29.30886615176194|
|  82678|29.267836075752573|
|  82879|29.260869697894222|
|  82460|29.218999332229416|
|  82298|29.210521264272458|
|  82780| 29.11393250585912|
|  82975|29.087137417260166|
|  82870|29.035611291845285|
|  82480|28.966630503571423|
|  82791|28.950074794315977|
|  82476|  28.9276052987726|
|  82590|28.869825836093444|
|  82578|28.858674607196377|
|  82493| 28.84008219335511|
|  82693|28.707614601018964|
|  82588| 28.68681230116714|
|  82564| 28.56894557892424|
|  82392|28.565107830433384|
+-------+------------------+
only showing top 20 rows



### Ejercicio

Cuantos casos hay por cada estacion, para cada una de las categorias creadas en `TempBulboSeco_cat`?

Escribir un filtro usando las funciones de spark, o spark SQL.